In [9]:
!pip install scikit-learn

  Using cached scikit_learn-1.5.1-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (12 kB)
  Using cached scipy-1.14.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (60 kB)
  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
Using cached scikit_learn-1.5.1-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (13.1 MB)
Using cached scipy-1.14.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (40.8 MB)
Using cached threadpoolctl-3.5.0-py3-none-any.whl (18 kB)


In [7]:
import nltk
nltk.download('gutenberg')
from nltk.corpus import gutenberg
import pandas as pd

data = gutenberg.raw('shakespeare-hamlet.txt')
with open('hamlet.txt','w') as file:
    file.write(data)

[nltk_data] Downloading package gutenberg to /home/suraj-
[nltk_data]     gupta/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


In [10]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

In [11]:
#load the dataset
with open('hamlet.txt', 'r') as file:
    text = file.read().lower()
    
#Tokenize the text
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index)+1
total_words

4818

In [18]:
#create input sequences
input_sequences=[]
for line in text.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence=token_list[:i+1]
        input_sequences.append(n_gram_sequence)

In [20]:
#Pad Sequences

max_sequence_len = max([len(x) for x in input_sequences])
max_sequence_len

14

In [21]:
input_sequences= np.array(pad_sequences(input_sequences,max_sequence_len))

In [22]:
input_sequences

array([[   0,    0,    0, ...,    0,    1,  687],
       [   0,    0,    0, ...,    1,  687,    4],
       [   0,    0,    0, ...,  687,    4,   45],
       ...,
       [   0,    0,    0, ...,    4,   45, 1047],
       [   0,    0,    0, ...,   45, 1047,    4],
       [   0,    0,    0, ..., 1047,    4,  193]], dtype=int32)

In [23]:
import tensorflow as tf
x,y = input_sequences[:,:-1],input_sequences[:,-1]

In [24]:
y=tf.keras.utils.to_categorical(y,num_classes=total_words)

In [25]:
y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [26]:
X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=0.2)

In [29]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

model = Sequential()
model.add(Embedding(total_words,100))
model.add(LSTM(150,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dense(total_words, activation="softmax"))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_4 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [30]:
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test), verbose=True)

Epoch 1/10
644/644 ━━━━━━━━━━━━━━━━━━━━ 20s 25ms/step - accuracy: 0.0308 - loss: 7.1341 - val_accuracy: 0.0365 - val_loss: 6.7674
Epoch 2/10
644/644 ━━━━━━━━━━━━━━━━━━━━ 21s 32ms/step - accuracy: 0.0374 - loss: 6.4576 - val_accuracy: 0.0383 - val_loss: 6.8408
Epoch 3/10
644/644 ━━━━━━━━━━━━━━━━━━━━ 23s 36ms/step - accuracy: 0.0428 - loss: 6.2996 - val_accuracy: 0.0517 - val_loss: 6.8732
Epoch 4/10
644/644 ━━━━━━━━━━━━━━━━━━━━ 23s 36ms/step - accuracy: 0.0536 - loss: 6.1526 - val_accuracy: 0.0544 - val_loss: 6.8950
Epoch 5/10
644/644 ━━━━━━━━━━━━━━━━━━━━ 24s 37ms/step - accuracy: 0.0531 - loss: 6.0295 - val_accuracy: 0.0618 - val_loss: 6.8973
Epoch 6/10
644/644 ━━━━━━━━━━━━━━━━━━━━ 23s 36ms/step - accuracy: 0.0596 - loss: 5.9176 - val_accuracy: 0.0647 - val_loss: 6.9633
Epoch 7/10
644/644 ━━━━━━━━━━━━━━━━━━━━ 23s 36ms/step - accuracy: 0.0696 - loss: 5.7451 - val_accuracy: 0.0682 - val_loss: 6.9926
Epoch 8/10
644/644 ━━━━━━━━━━━━━━━━━━━━ 24s 37ms/step - accuracy: 0.0778 - loss: 5.6010 - 

In [31]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ (None, 13, 100)        │       481,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ (None, 13, 150)        │       150,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 13, 150)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_4 (LSTM)                   │ (None, 100)            │       100,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4818)           │       486,618 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,658,256 (13.96 MB)

 Trainable params: 1,219,418 (4.65 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,438,838 (9.30 MB)

In [34]:
#Function to predict the next word

def predict_next_word(model,tokensizer,text,max_sequence_len):
    token_list=tokenizer.texts_to_sequences([text])[0]
    if len(token_list) >= max_sequence_len:
        token_list=token_list[-(max_sequence_len-1):]
    token_list=pad_sequences([token_list], maxlen=max_sequence_len)
    predicted = model.predict(token_list,verbose=0)
    predicted_word_index=np.argmax(predicted, axis=1)
    for word, index in tokenizer.word_index.items():
        if index == predicted_word_index:
            return word
    return None

In [43]:
input_text="Barn. In the same figure, like the King that's"
max_sequence_len=model.input_shape[1]+1
next_word = predict_next_word(model, tokenizer, input_text, max_sequence_len)

In [44]:
print(next_word)

the
